# First file for learning Jupyter
This is the first local file that I made to learn how to work iwth jupyter. 

In [1]:
from csv import reader
# open files
google_store_link, apple_store_link = (open('raw_data/googleplaystore.csv', encoding="utf8"),
                                       open('raw_data/AppleStore.csv', encoding="utf8"))
# read the opened files
google_store, apple_store = (reader(google_store_link), 
                             reader(apple_store_link))
# convert the opened files into lists that are useable
apps_g, apps_a = (list(google_store), 
                  list(apple_store))
# extract data
apps_google, apps_apple = (apps_g[1:], 
                           apps_a[1:])
# extract header
header_google, header_apple = (apps_g[0], 
                               apps_a[0])


# Write an explore function

And glance at the data set to ensure the data is parsed as it is supposed to be. 


In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
explore_data(apps_google, 0, 3, True)
print("\n")
explore_data(apps_apple, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', '15-Jan-18', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0', '2974676', '212', '3.5', '3.5', '95', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0', '2161558', '1289', '4.5', '4', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '

# Data cleaning

## Objectives

1. Only retain apps that are free
2. Remove non-english apps


In [3]:
print(apps_google[10471:10474])

[['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', '5-Aug-14', '3.0.0', '4.0.3 and up'], ['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', '11-Feb-18', '1.0.19', '4.0 and up', ''], ['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', '7-Aug-18', '6.06.14', '4.4 and up']]


Printing of the row 10472 along with a row above abd below indicates that there is an extra entry in the list at the end. Next, we remove it, instead of deleting the entire line. 

In [4]:
if len(apps_google[10472]) == 13:
    del apps_google[10472]

In [5]:
print(apps_google[10471:10474])

[['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', '5-Aug-14', '3.0.0', '4.0.3 and up'], ['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', '7-Aug-18', '6.06.14', '4.4 and up'], ['Sat-Fi Voice', 'COMMUNICATION', '3.4', '37', '14M', '1,000+', 'Free', '0', 'Everyone', 'Communication', '21-Nov-14', '2.2.1.5', '2.2 and up']]


# Removed duplicate app entries

If we examine the rows main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show the data was collected at different times. We ce can use this information to build a criterion for removing the duplicates. The higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given a
pp.

In [6]:
duplicate_apps = []
unique_apps = []

for row in apps_google:
    name = row[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


In [7]:
print("The header of the google dataset is: ", header_google)
print("The header of the apple dataset is: ", header_apple)

The header of the google dataset is:  ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
The header of the apple dataset is:  ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Create a dictionary that has all the unique app names as the keys and the ratings as the values associated with those keys. 

This dictionary will help us remove the duplicate data in next cell. 

In [8]:
reviews_max = {}

for row in apps_google:
    name = row[0]
    n_reviews = float(row[3])

    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews
print(len(reviews_max))

        

9659


Next cell we select only the rows that contain the data values tha match tha values in the dictionary. 

In [9]:
google_clean = []
already_added = []

for row in apps_google:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        google_clean.append(row)
        already_added.append(name)
explore_data(google_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', '8-Jun-18', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


# Removed non-English apps

The following code cell makes a function to test if the app name has at least 3 instances of  ASCII characters less than  or equal to 127. Everything else is non typical character. 

In [10]:
def acs_127(input_string):
    counter = 0
    for character in input_string:
        if ord(character) > 127:
            counter += 1
    if counter > 3:
        return False
    else:
            return True

In [11]:
print(acs_127(apps_apple[814][1]))

print(acs_127('Instagram'))
print(acs_127('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(acs_127('Docs To Go™ Free Office Suite'))
print(acs_127('Instachat 😜'))

True
True
False
True
True


In [12]:
apps_apple_eng = []
apps_google_eng = []

for row in apps_apple:
    app_name = row[0]
    if acs_127(app_name):
        apps_apple_eng.append(row)

for row in google_clean:
    app_name = row[0]
    if acs_127(app_name):
        apps_google_eng.append(row)

explore_data(apps_apple_eng, 0, 3, True)

explore_data(apps_google_eng, 0, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0', '2974676', '212', '3.5', '3.5', '95', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0', '2161558', '1289', '4.5', '4', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', '8-Jun-18', 'Varies with device', '4.2 and up'

# Filter free apps

In [13]:
apps_apple_eng_free = []
apps_google_eng_free = []

for row in apps_apple_eng:
    price = row[4]
    if price == "0":
        apps_apple_eng_free.append(row)

for row in apps_google_eng:
    price = row[7]
    if price == "0":
        apps_google_eng_free.append(row)

explore_data(apps_apple_eng_free, 0, 0, True)

explore_data(apps_google_eng_free, 0, 0, True)

Number of rows: 4056
Number of columns: 16
Number of rows: 8864
Number of columns: 13


# Most common apps by genre
Our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

To minimize risks and overhead, our validation strategy for an app idea has three steps:1. 

Build a minimal Android version of the app, and add it to Google Pl2. ay.
If the app has a good response from users, we develop it furt3. her.
If the app is profitable after six months, we build an iOS version of the app and add it to the App 

tore.
Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful in both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

In [14]:
def freq_table(df, index):
    table = {}
    total = 0
    
    for row in df:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages


apple_app_category = freq_table(apps_apple_eng_free, 11)
google_app_category = freq_table(apps_google_eng_free, 1)

print(apple_app_category)
print(google_app_category)

{'Social Networking': 3.5256410256410255, 'Photo & Video': 4.117357001972387, 'Games': 55.64595660749507, 'Music': 1.6518737672583828, 'Reference': 0.4930966469428008, 'Health & Fitness': 1.8737672583826428, 'Weather': 0.7642998027613412, 'Utilities': 2.687376725838264, 'Travel': 1.3806706114398422, 'Shopping': 2.983234714003945, 'News': 1.4299802761341223, 'Navigation': 0.4930966469428008, 'Lifestyle': 2.3175542406311638, 'Entertainment': 8.234714003944774, 'Food & Drink': 1.0601577909270217, 'Sports': 1.947731755424063, 'Book': 1.6272189349112427, 'Finance': 2.0710059171597637, 'Education': 3.2544378698224854, 'Productivity': 1.5285996055226825, 'Business': 0.4930966469428008, 'Catalogs': 0.22189349112426035, 'Medical': 0.19723865877712032}
{'ART_AND_DESIGN': 0.6430505415162455, 'AUTO_AND_VEHICLES': 0.9250902527075812, 'BEAUTY': 0.5979241877256317, 'BOOKS_AND_REFERENCE': 2.1435018050541514, 'BUSINESS': 4.591606498194946, 'COMICS': 0.6204873646209386, 'COMMUNICATION': 3.23781588447653

In [15]:
def display_table(df, index):
    table = freq_table(df, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

display_table(apps_apple_eng_free, 11)
print("\n")
display_table(apps_google_eng_free, 1)
print("\n")
display_table(apps_google_eng_free, -4)

Games : 55.64595660749507
Entertainment : 8.234714003944774
Photo & Video : 4.117357001972387
Social Networking : 3.5256410256410255
Education : 3.2544378698224854
Shopping : 2.983234714003945
Utilities : 2.687376725838264
Lifestyle : 2.3175542406311638
Finance : 2.0710059171597637
Sports : 1.947731755424063
Health & Fitness : 1.8737672583826428
Music : 1.6518737672583828
Book : 1.6272189349112427
Productivity : 1.5285996055226825
News : 1.4299802761341223
Travel : 1.3806706114398422
Food & Drink : 1.0601577909270217
Weather : 0.7642998027613412
Reference : 0.4930966469428008
Navigation : 0.4930966469428008
Business : 0.4930966469428008
Catalogs : 0.22189349112426035
Medical : 0.19723865877712032


FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084

# Most Popular Apps by Genre on the App Store


In [16]:
for genre in apple_app_category:
    total = 0 
    len_genre = 0
    for row in apps_apple_eng_free:
        genre_app = row[11]
        if genre_app == genre:
            user_rating = row[5]
            total += float(user_rating)
            len_genre += 1
    average_user_rating = total / len_genre
    print(genre, total, " : ", average_user_rating)

Social Networking 7590182.0  :  53078.195804195806
Photo & Video 4550732.0  :  27249.892215568863
Games 42713023.0  :  18924.68896765618
Music 3784296.0  :  56482.02985074627
Reference 1348958.0  :  67447.9
Health & Fitness 1516376.0  :  19952.315789473683
Weather 1463849.0  :  47220.93548387097
Utilities 1527101.0  :  14010.100917431193
Travel 1132097.0  :  20216.01785714286
Shopping 2268348.0  :  18746.677685950413
News 921778.0  :  15892.724137931034
Navigation 519441.0  :  25972.05
Lifestyle 843961.0  :  8978.308510638299
Entertainment 3614869.0  :  10822.961077844311
Food & Drink 867701.0  :  20179.093023255813
Sports 1590189.0  :  20128.974683544304
Book 560890.0  :  8498.333333333334
Finance 1135870.0  :  13522.261904761905
Education 827156.0  :  6266.333333333333
Productivity 1181341.0  :  19053.887096774193
Business 127356.0  :  6367.8
Catalogs 16016.0  :  1779.5555555555557
Medical 3678.0  :  459.75


Profile recommendation will require us to sort this data. WIll tackle later. 

In [18]:
print(header_google)


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [32]:
# print(display_table(apps_google_eng_free, 1))
google_category_installs = {}
for category in google_app_category:
    total = 0
    len_category = 0
    
    for row in apps_google_eng_free:
        category_app = row[1]
        if category_app == category:
            installs = row[5]
            installs = installs.replace("+", "")
            installs = installs.replace(",", "")
            installs = float(installs)
            total += installs
            len_category += 1

    avg_installs = total / len_category
    google_category_installs[category] = avg_installs
print(google_category_installs)
           

{'ART_AND_DESIGN': 1986335.0877192982, 'AUTO_AND_VEHICLES': 647317.8170731707, 'BEAUTY': 513151.88679245283, 'BOOKS_AND_REFERENCE': 8767811.894736841, 'BUSINESS': 1712290.1474201474, 'COMICS': 817657.2727272727, 'COMMUNICATION': 38456119.167247385, 'DATING': 854028.8303030303, 'EDUCATION': 1833495.145631068, 'ENTERTAINMENT': 11640705.88235294, 'EVENTS': 253542.22222222222, 'FINANCE': 1387692.475609756, 'FOOD_AND_DRINK': 1924897.7363636363, 'HEALTH_AND_FITNESS': 4188821.9853479853, 'HOUSE_AND_HOME': 1331540.5616438356, 'LIBRARIES_AND_DEMO': 638503.734939759, 'LIFESTYLE': 1437816.2687861272, 'GAME': 15588015.603248259, 'FAMILY': 3695641.8198090694, 'MEDICAL': 120550.61980830671, 'SOCIAL': 23253652.127118643, 'SHOPPING': 7036877.311557789, 'PHOTOGRAPHY': 17840110.40229885, 'SPORTS': 3638640.1428571427, 'TRAVEL_AND_LOCAL': 13984077.710144928, 'TOOLS': 10801391.298666667, 'PERSONALIZATION': 5201482.6122448975, 'PRODUCTIVITY': 16787331.344927534, 'PARENTING': 542603.6206896552, 'WEATHER': 50